In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue May 19 11:54:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls -al drive/My\ Drive/Facebook_bAbI

total 1290089
-rw------- 1 root root     186535 May 19 11:36  Bert.ipynb
-rw------- 1 root root 1314604232 May 18 20:38  best_bert_base_task5-2_12_classW.hdf5
-rw------- 1 root root      18393 May 19 11:54  ChatBot.ipynb
-rw------- 1 root root     228551 May 19 11:28 'Copie a fișierului Bert.ipynb'
-rw------- 1 root root      31743 May 19 11:28 'Copie a fișierului ChatBot.ipynb'
-rw------- 1 root root     535231 Jun  9  2016  dialog-babi-kb-all.txt
-rw------- 1 root root    2723622 May 18 20:19  task5_dev_trn.csv
-rw------- 1 root root    1352677 May 18 20:19  task5_tst.csv
-rw------- 1 root root    1368267 May 18 20:19  task5_tst_OOV.csv


In [4]:
!rm tokenization.py*
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install sentencepiece
import tokenization

rm: cannot remove 'tokenization.py*': No such file or directory
     |████████████████████████████████| 1.1MB 6.6MB/s 


In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/working'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import tensorflow_hub as hub
import keras.backend as kb
from sklearn.utils import class_weight
import keras
import pprint
pp = pprint.PrettyPrinter(indent=4)

Using TensorFlow backend.


In [0]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2"

In [0]:
bert_layer = hub.KerasLayer(module_url, trainable=True)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [0]:
from collections import defaultdict

def get_restaurants_dict(file_name):
  restaurants = defaultdict(dict)
  with open (file_name, 'r') as content_file:
    content = content_file.read()
    lines = content.split("\n")[:-1]

    for line in lines:
      words = line.split()
      restaurants[words[1]][words[2]] = words[3]
    
  return restaurants

restaurants = get_restaurants_dict('drive/My Drive/Facebook_bAbI/dialog-babi-kb-all.txt')

#pp.pprint(restaurants)

def get_restaurants(location, cuisine, number_people, price):
  result = []
  for name in restaurants.keys():
    if restaurants[name]["R_location"] == location and \
       restaurants[name]["R_cuisine"] == cuisine and \
       restaurants[name]["R_number"] == number_people and \
       restaurants[name]["R_price"] == price:
       result.append((name, restaurants[name]))
  return sorted(result, key = lambda x: x[1]["R_rating"])

#pp.pprint(get_restaurants("madrid", "spanish", "two", "cheap"))


In [0]:
def bert_encode(texts, tokenizer, max_len=512, use_continuous_segment_ids = False):
    all_ids = []
    all_masks = []
    all_segments = []
    maximum_len = 0

    for sentences in texts:
      input_sequence = ["[CLS]"]
      pad_masks = [0]
      segment_ids = [0]

      question = True
      #print('sentences.split("#")[:-1]', sentences.split("#")[:-1])
      for sentence in sentences.split("#")[:-1]:
        #print("[", sentence, "]")
        if question == True:
          pad = 0
          question = False
        else:
          pad = 1
          question = True
     
        tokens = tokenizer.tokenize(sentence)
        input_sequence += tokens 
        input_sequence += ["[SEP]"]

        #segment_ids += range(1, len(tokens) + 2)
  
        pad_masks += [pad]*(len(tokens) + 1)


      
      input_sequence = input_sequence[:max_len]
       
      pad_len = max_len - len(input_sequence)
      
      ids = tokenizer.convert_tokens_to_ids(input_sequence)
      maximum_len = max(maximum_len, len(ids))
      ids += [0]*pad_len
      #pad_masks += [0]*pad_len
      #segment_ids = [i for i in range(len(tokens))]
      pad_masks = [1] * len(input_sequence) + [0] * pad_len
      segment_ids = [0]*max_len

      if use_continuous_segment_ids:
        segment_ids = range(max_len)

      if len(ids) != max_len or len(pad_masks) != max_len or len(segment_ids) != max_len:
        print("len(ids)", len(ids))
        print("len(pad_masks)", len(pad_masks))
        print("len(segment_ids)", len(segment_ids))
        print()

      all_ids.append(ids)
      all_masks.append(pad_masks)
      all_segments.append(segment_ids)

      
      """
      print("len(input_sequence)", len(input_sequence))
      print("len(ids)", len(ids))
      print("len(pad_masks)", len(pad_masks))
      print("len(segment_ids)", len(segment_ids))

      print("input_sequence",input_sequence)
      print("ids", ids)
      print("pad_masks", pad_masks)
      print("segment_ids", segment_ids)

      for i in range(max_len):
        print("pad_masks: ", pad_masks[i], " segment_ids: ", segment_ids[i], " token: ", input_sequence[i] if i < len(input_sequence) else "N/A ", " id: ", ids[i] )

      print()
      """
      
    #print("maximum_len", maximum_len)
    return np.array(all_ids), np.array(all_masks), np.array(all_segments)


In [0]:
def build_model(bert_layer, max_len=512, max_out_len=16):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dropout(0.2) (clf_output)
    out = Dense(max_out_len, activation='softmax')(out)
    
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss=tf.keras.losses.categorical_crossentropy, metrics=["categorical_accuracy"])
    
    return model

In [11]:
model = build_model(bert_layer, max_len=100 , max_out_len = 12)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [0]:
model.load_weights('drive/My Drive/Facebook_bAbI/best_bert_base_task5-2_12_classW.hdf5')

In [0]:
locations = ['paris', 'london', 'bombay', 'madrid', 'rome', 'seoul', 'tokyo', 
             'beijing',  'bangkok', 'hanoi']
cuisines = ['italian', 'british', 'indian', 'spanish', 'french', 'thai', 
            'korean', 'cantonese', 'vietnamese', 'japanese']
numbers_people = ['two', 'eight', 'six', 'four']   
prices = ['cheap', 'moderate', 'expensive']

def get_params(conv):
  location = None
  cuisine = None
  number_people = None
  price = None

  for word in conv.lower().split():
    #print(word)
    if word in locations:
      location = word
    if word in cuisines:
      cuisine = word

    if word in numbers_people:
      number_people = word

    if word in prices:
      price = word
  """
  print("location", location)
  print("cuisine", cuisine)
  print("number_people", number_people)
  print("price", price)
  """
  return location, cuisine, number_people, price
    


In [14]:
get_params("i want a table in Paris # any preference on a type of cuisine # i think italian # where should it be # in paris # which price range are looking for # expensive #")

('paris', 'italian', None, 'expensive')

In [0]:
reverse =  {   0: 'where should it be',
                1: 'how many people would be in your party',
                2: 'which price range are looking for',
                3: 'api_call',
                4: 'sure is there anything else to update',
                5: 'sure let me find an other option for you',
                6: 'great let me do the reservation',
                7: 'phone_call',
                8: 'is there anything i can help you with',
                9: "you're welcome",
                10: 'any preference on a type of cuisine',
                11: '_address'}
pretty_reverse =  {   0: 'Where should it be?',
                1: 'How many people would be in your party?',
                2: 'Which price range are looking for?',
                3: 'api_call',
                4: 'Sure. Is there anything else to update?',
                5: 'Sure, let me find an other option for you.',
                6: 'Great, let me do the reservation.',
                7: 'phone_call',
                8: 'Is there anything i can help you with?',
                9: "You're welcome.",
                10: 'Any preference on a type of cuisine?',
                11: '_address'}

In [18]:
def resp(conv):
  print("conv", conv)
  conv_input = bert_encode([conv], tokenizer, max_len=300)
  #print(conv_input)

  pred = model.predict(conv_input)
  print("pred.argmax(axis=1)", pred.argmax(axis=1))
  answer  = pred.argmax(axis=1)[0]

  reply = "N/A"
  if answer in [0, 1, 2, 10]:
    reply = rev_answers[answer]
  return reply

max_len = 100
def main():
    statement = input("> ")
    reply = "Hello, what can i help you with today?"
    print (" "*(max_len - len(reply)),reply)

    conv = ""
    prev_conv = ""
    options = None
    debug = ""
    while True:
        #print("[ conv" , conv, "]")
        debug += conv + "\n"
        statement = input("> ")
        if statement == "print debug":
          print(debug)

        conv += statement + " # "
        prev_conv += statement + " # "

        #print("conv", conv)
        conv_input = bert_encode([conv], tokenizer, max_len=100)
        #print(conv_input)

        pred = model.predict(conv_input)
        #print("pred.argmax(axis=1)", pred.argmax(axis=1))
        debug += str(pred.argmax(axis=1)) + "\n"
        answer  = pred.argmax(axis=1)[0]

        conv += reverse[answer] + " # " 

        reply = "N/A"
        
        if answer in [0, 1, 2, 4, 6, 8, 9, 10]:
          reply = pretty_reverse[answer]
          print (" "*(max_len - len(reply)),reply)
          if answer == 9:
            return           
        elif answer == 3:
          location, cuisine, number_people, price = get_params(prev_conv.replace("?", ""))
          #print("[api_call ", location, cuisine, number_people, price, "]")
          debug += str(location) + str(cuisine) + str(number_people) + str(price) + "\n"
          options = get_restaurants(location, cuisine, number_people, price)
          #pp.pprint(options)
          debug += str(options) + "\n"
          higest_r_restaurant_name = options[-1][0]
          reply = "what do you think of this option:" + \
                  higest_r_restaurant_name + "?"
          print (" "*(max_len - len(reply)),reply)
        elif answer == 5:
          options.pop()
          if len(options) == 0:
            print("There are no more restaurants")
            return
          higest_r_restaurant_name = options[-1][0]
          reply = "what do you think of this option:" + \
                higest_r_restaurant_name + "?"
          print (" "*(max_len - len(reply)),reply)
        elif answer == 7: # phone_call
          reply = "Here it is: "  + options[-1][1]["R_phone"]
          print (" "*(max_len - len(reply)),reply)
        elif answer == 11:
          reply = "Here it is: " + options[-1][1]["R_address"]
          print (" "*(max_len - len(reply)),reply)
        else:
          print("Unkownd answer", pred.argmax(axis=1))

        #print("[ conv" , conv, "]")
        debug += conv + "\n"
        if reverse[answer] in ["sure is there anything else to update",
                              "sure let me find an other option for you",
                              "great let me do the reservation",
                              "is there anything i can help you with",
                               "api_call"]:
          conv = ""
                        
        
          debug += "\n"


if __name__ == "__main__":
    main()

> Hello
                                                               Hello, what can i help you with today?
>  may i have a table for four
                                                                 Any preference on a type of cuisine?
> with spanish cuisine
                                                                                  Where should it be?
> in bombay
                                                                   Which price range are looking for?
> in a expensive price range please
                              what do you think of this option:resto_bombay_expensive_spanish_8stars?
> i like it
                                                                    Great, let me do the reservation.
> may i have the phone number of the restaurant?
                                              Here it is: resto_bombay_expensive_spanish_8stars_phone
> thanks
                                                               Is there anything i can help you with?
> no

In [0]:
pp.pprint(get_restaurants("paris", "british", "six", "expensive"))